In [2]:
#导入程序包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import tushare as ts #主要数据来源
import requests
import re
import time as t
import random
from scipy.stats.mstats import winsorize
from wordcloud import WordCloud
import seaborn as sns
from sklearn.cluster import KMeans
from scipy import stats
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from heamy.dataset import Dataset
from heamy.estimator import Classifier
from heamy.pipeline import ModelsPipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
import xgboost as xgb
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import roc_auc_score
#from sklearn.metrics import plot_roc_curve   
from sklearn.metrics import roc_curve
from collections import Counter
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter('ignore')

In [ ]:
def craw(gb_id):
    cookies = {
        'qgqp_b_id': '9aafee8f9867edbe280a3884dea14c1d',
        'st_si': '39434491176058',
        'st_asi': 'delete',
        'show_app_box_time': '1672224935454',
        'st_pvi': '96267766858191',
        'st_sp': '2022-12-28%2018%3A39%3A33',
        'st_inirUrl': 'http%3A%2F%2Fguba.eastmoney.com%2F',
        'st_sn': '59',
        'st_psi': '20221228190006487-117016304298-2296127990',
    }
    
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Origin': 'http://mguba.eastmoney.com',
        'Referer': 'http://mguba.eastmoney.com/mguba/list/600000,f_1',
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36',
    }
    

    
    
    page=1
    
    cd=1
    lst=[]
    while cd>0:
      
        # time.sleep(1)
        
        # print(page)
        
        url='http://mguba.eastmoney.com/mguba2020/interface/GetData.aspx'

        data = {
          'param': f'code={gb_id}&p={page}&ps=20&sorttype=0',
          'plat': 'wap',
          'version': '200',
          'path': '/webexparticlelist/api/article/articlelist',
          'env': '1',
          'origin': '',
          'ctoken': '',
          'utoken': ''
        }
        try:
            response = requests.post(url, headers=headers, cookies=cookies, data=data, verify=False)
            
            sj=response.json()['re']
            
            cd=len(sj)
            
            # print(page)       
            
            try:
                end_post_publish_time=sj[-1]['post_publish_time']
            except:
                end_post_publish_time=''
            
            if '2023' in str(end_post_publish_time) or '2022-12' in str(end_post_publish_time):
            # if '2023' in str(end_post_publish_time):
                
                
                # print(page)
                
                for i in sj:
                    # post_publish_time=i.get("post_publish_time")
                    
                    
                    dic={}
                    dic['gb_id']=gb_id
                    dic['帖子时间']=i.get("post_publish_time")
                    post_title=i.get("post_title")
                    
                    if post_title:
                        
                        dic['帖子标题']=post_title
                    else:
                        dic['帖子标题']=i.get("post_content")
                    post_id=i.get("post_id")
                    dic['帖子URL']=f'http://guba.eastmoney.com/news,{gb_id},{post_id}.html'
                    dic['帖子作者']=i.get("post_user").get("user_nickname")
                    dic['浏览量']=i.get("post_click_count")
                    dic['评论量']=i.get("post_comment_count")
                    dic['点赞量']=i.get("post_like_count")
                    lst.append(dic)
                    print(gb_id,dic['帖子时间'])
                df=pd.DataFrame(lst)
                df.drop_duplicates(subset=['帖子URL'],keep='first',inplace=True)
                df.to_csv(str(gb_id)+'.csv',index=None,encoding='utf-8-sig')            

            else:
                break
            page=page+1
            print(page)
        except Exception as e:
            print(e)
        
            


            
basic_stock=pd.read_excel('股票代码和名字.xlsx',dtype='str')


for k in list(basic_stock['code']):
    
    craw(k)
          
import pandas as pd
ls = []
lst = pd.read_excel('股票代码和名字.xlsx',dtype='str')['code']

for i in lst:
    sj = pd.read_csv(i+'.csv')
    sj['年月'] = sj['帖子时间'].str[0:7]
    cond = sj['年月'] == '2022-12'
    sj2 = pd.DataFrame(sj[cond]['帖子标题'])
    sj2 = sj2.reset_index()
    del sj2['index']
    sj2.rename(columns={"帖子标题": i.split('.')[0]},inplace=True)
    ls.append(sj2)

comments = pd.concat(ls,axis = 1)

000676 2023-02-18 09:58:38
000676 2023-02-18 09:52:43
000676 2023-02-18 09:42:18
000676 2023-02-18 09:35:05
000676 2023-02-18 09:30:06
000676 2023-02-18 08:23:26
000676 2023-02-18 07:42:35
000676 2023-02-18 00:14:26
000676 2023-02-18 00:07:40
000676 2023-02-17 22:42:08
000676 2023-02-17 22:08:28
000676 2023-02-17 20:58:36
000676 2023-02-17 20:56:05
000676 2023-02-17 20:31:24
000676 2023-02-17 20:28:19
000676 2023-02-17 19:55:37
000676 2023-02-17 19:39:31
000676 2023-02-17 18:47:28
000676 2023-02-17 18:39:44
000676 2023-02-17 18:31:03
2
000676 2023-02-17 18:12:26
000676 2023-02-17 18:05:05
000676 2023-02-17 17:16:21
000676 2023-02-17 17:08:42
000676 2023-02-17 17:01:32
000676 2023-02-17 16:48:45
000676 2023-02-17 16:40:05
000676 2023-02-17 16:29:06
000676 2023-02-17 16:20:24
000676 2023-02-17 16:19:18
000676 2023-02-17 16:02:00
000676 2023-02-17 15:56:49
000676 2023-02-17 15:39:54
000676 2023-02-17 15:32:50
000676 2023-02-17 15:32:01
000676 2023-02-17 15:25:38
000676 2023-02-17 15:24:20

000676 2023-02-16 12:55:04
000676 2023-02-16 12:52:36
000676 2023-02-16 12:50:37
000676 2023-02-16 12:32:00
000676 2023-02-16 12:30:16
000676 2023-02-16 11:57:58
000676 2023-02-16 11:57:54
000676 2023-02-16 11:56:13
000676 2023-02-16 11:51:02
000676 2023-02-16 11:42:35
000676 2023-02-16 11:42:15
000676 2023-02-16 11:34:38
000676 2023-02-16 11:34:18
000676 2023-02-16 11:30:18
000676 2023-02-16 11:29:54
000676 2023-02-16 11:29:29
000676 2023-02-16 11:24:53
18
000676 2023-02-16 11:18:23
000676 2023-02-16 11:18:02
000676 2023-02-16 11:15:52
000676 2023-02-16 11:15:40
000676 2023-02-16 11:15:14
000676 2023-02-16 11:12:37
000676 2023-02-16 11:10:13
000676 2023-02-16 11:06:54
000676 2023-02-16 11:02:56
000676 2023-02-16 11:02:35
000676 2023-02-16 10:59:12
000676 2023-02-16 10:49:12
000676 2023-02-16 10:48:05
000676 2023-02-16 10:47:38
000676 2023-02-16 10:47:33
000676 2023-02-16 10:47:22
000676 2023-02-16 10:43:06
000676 2023-02-16 10:35:41
000676 2023-02-16 10:31:03
000676 2023-02-16 10:30:1

33
000676 2023-02-15 00:39:01
000676 2023-02-15 00:07:43
000676 2023-02-14 23:19:57
000676 2023-02-14 22:44:32
000676 2023-02-14 21:48:09
000676 2023-02-14 21:43:42
000676 2023-02-14 20:42:43
000676 2023-02-14 20:25:51
000676 2023-02-14 19:29:06
000676 2023-02-14 19:08:20
000676 2023-02-14 19:00:39
000676 2023-02-14 18:57:36
000676 2023-02-14 18:42:35
000676 2023-02-14 17:30:40
000676 2023-02-14 16:44:56
000676 2023-02-14 15:34:15
000676 2023-02-14 15:14:56
000676 2023-02-14 15:06:11
000676 2023-02-14 15:03:31
000676 2023-02-14 15:03:08
34
000676 2023-02-14 15:01:17
000676 2023-02-14 14:56:50
000676 2023-02-14 14:54:13
000676 2023-02-14 14:53:27
000676 2023-02-14 14:50:11
000676 2023-02-14 14:43:51
000676 2023-02-14 14:35:54
000676 2023-02-14 14:35:00
000676 2023-02-14 14:33:58
000676 2023-02-14 14:15:53
000676 2023-02-14 14:15:25
000676 2023-02-14 14:10:38
000676 2023-02-14 14:01:22
000676 2023-02-14 13:53:31
000676 2023-02-14 13:38:06
000676 2023-02-14 13:31:53
000676 2023-02-14 13:2

000676 2023-02-09 10:48:22
000676 2023-02-09 10:46:44
000676 2023-02-09 10:46:35
000676 2023-02-09 10:46:27
000676 2023-02-09 10:45:34
000676 2023-02-09 10:44:43
000676 2023-02-09 10:43:30
000676 2023-02-09 10:42:48
000676 2023-02-09 10:42:09
000676 2023-02-09 10:40:22
000676 2023-02-09 10:36:11
000676 2023-02-09 10:35:09
000676 2023-02-09 10:33:43
000676 2023-02-09 10:33:28
49
000676 2023-02-09 10:33:16
000676 2023-02-09 10:33:01
000676 2023-02-09 10:32:50
000676 2023-02-09 10:32:30
000676 2023-02-09 10:32:10
000676 2023-02-09 10:31:54
000676 2023-02-09 10:31:45
000676 2023-02-09 10:31:34
000676 2023-02-09 10:29:40
000676 2023-02-09 10:29:18
000676 2023-02-09 10:28:44
000676 2023-02-09 10:27:37
000676 2023-02-09 10:27:26
000676 2023-02-09 10:27:18
000676 2023-02-09 10:26:42
000676 2023-02-09 10:26:34
000676 2023-02-09 10:26:11
000676 2023-02-09 10:25:25
000676 2023-02-09 10:24:30
000676 2023-02-09 10:24:20
50
000676 2023-02-09 10:23:57
000676 2023-02-09 10:22:49
000676 2023-02-09 10:2

000676 2023-02-02 10:48:44
000676 2023-02-02 10:22:47
000676 2023-02-02 10:11:19
000676 2023-02-02 10:06:46
000676 2023-02-02 10:03:29
000676 2023-02-02 09:54:13
000676 2023-02-02 08:43:16
000676 2023-02-02 07:53:26
000676 2023-02-02 07:15:57
000676 2023-02-02 06:33:01
000676 2023-02-02 00:06:45
000676 2023-02-01 22:43:02
000676 2023-02-01 21:50:04
000676 2023-02-01 21:08:27
000676 2023-02-01 19:22:32
65
000676 2023-02-01 19:11:24
000676 2023-02-01 18:12:50
000676 2023-02-01 17:54:24
000676 2023-02-01 17:10:34
000676 2023-02-01 17:06:14
000676 2023-02-01 17:06:12
000676 2023-02-01 17:03:59
000676 2023-02-01 17:02:32
000676 2023-02-01 16:56:33
000676 2023-02-01 15:32:47
000676 2023-02-01 15:15:35
000676 2023-02-01 15:15:09
000676 2023-02-01 15:00:13
000676 2023-02-01 14:22:49
000676 2023-02-01 14:06:38
000676 2023-02-01 13:53:26
000676 2023-02-01 13:48:43
000676 2023-02-01 13:46:24
000676 2023-02-01 13:36:54
000676 2023-02-01 13:28:54
66
000676 2023-02-01 13:08:38
000676 2023-02-01 13:0

...

000736 2023-01-06 09:29:41
000736 2023-01-06 09:29:12
000736 2023-01-06 09:29:06
000736 2023-01-06 09:28:31
000736 2023-01-06 09:27:44
000736 2023-01-06 09:27:43
000736 2023-01-06 09:26:32
000736 2023-01-06 09:25:02
000736 2023-01-06 09:24:37
000736 2023-01-06 09:21:58
000736 2023-01-06 09:21:53
000736 2023-01-06 09:20:44
000736 2023-01-06 09:16:01
000736 2023-01-06 09:14:33
000736 2023-01-06 08:53:14
000736 2023-01-06 08:48:06
000736 2023-01-06 08:38:47
000736 2023-01-06 08:35:06
000736 2023-01-06 08:33:42
000736 2023-01-06 08:24:17
175
000736 2023-01-06 07:42:37
000736 2023-01-06 07:35:17
000736 2023-01-06 06:08:54
000736 2023-01-06 05:41:04
000736 2023-01-06 03:23:33
000736 2023-01-06 01:48:09
000736 2023-01-06 01:30:14
000736 2023-01-06 00:35:51
000736 2023-01-06 00:07:21
000736 2023-01-05 23:55:26
000736 2023-01-05 23:29:24
000736 2023-01-05 23:15:41
000736 2023-01-05 23:05:34
000736 2023-01-05 22:39:23
000736 2023-01-05 22:22:19
000736 2023-01-05 22:15:44
000736 2023-01-05 22:13:

000736 2023-01-04 14:58:23
000736 2023-01-04 14:58:04
000736 2023-01-04 14:57:46
000736 2023-01-04 14:57:05
000736 2023-01-04 14:57:02
000736 2023-01-04 14:56:20
000736 2023-01-04 14:55:55
000736 2023-01-04 14:53:56
000736 2023-01-04 14:53:53
000736 2023-01-04 14:53:16
000736 2023-01-04 14:53:01
000736 2023-01-04 14:52:56
000736 2023-01-04 14:52:52
190
000736 2023-01-04 14:51:37
000736 2023-01-04 14:50:03
000736 2023-01-04 14:49:08
000736 2023-01-04 14:48:48
000736 2023-01-04 14:48:37
000736 2023-01-04 14:43:52
000736 2023-01-04 14:43:20
000736 2023-01-04 14:42:16
000736 2023-01-04 14:42:08
000736 2023-01-04 14:40:22
000736 2023-01-04 14:38:31
000736 2023-01-04 14:34:10
000736 2023-01-04 14:23:11
000736 2023-01-04 14:21:22
000736 2023-01-04 14:20:29
000736 2023-01-04 14:19:49
000736 2023-01-04 14:17:22
000736 2023-01-04 14:16:50
000736 2023-01-04 14:16:25
000736 2023-01-04 14:14:33
191
000736 2023-01-04 14:13:52
000736 2023-01-04 14:11:45
000736 2023-01-04 14:11:29
000736 2023-01-04 14

000736 2023-01-03 08:50:57
000736 2023-01-03 00:09:14
000736 2023-01-02 22:50:17
000736 2023-01-02 22:36:43
000736 2023-01-02 22:32:15
000736 2023-01-02 22:09:17
000736 2023-01-02 21:45:18
000736 2023-01-02 19:25:42
000736 2023-01-02 19:10:34
000736 2023-01-02 19:08:27
000736 2023-01-02 18:04:12
000736 2023-01-02 18:02:28
000736 2023-01-02 16:51:33
000736 2023-01-02 16:26:39
000736 2023-01-02 15:09:54
000736 2023-01-02 13:50:39
000736 2023-01-02 13:21:20
000736 2023-01-02 12:34:55
000736 2023-01-02 12:04:06
000736 2023-01-02 10:28:58
206
000736 2023-01-02 09:41:00
000736 2023-01-02 02:46:17
000736 2023-01-01 21:13:26
000736 2023-01-01 20:55:23
000736 2023-01-01 19:44:02
000736 2023-01-01 17:50:39
000736 2023-01-01 17:07:40
000736 2023-01-01 17:05:56
000736 2023-01-01 16:09:14
000736 2023-01-01 15:40:04
000736 2023-01-01 14:38:34
000736 2023-01-01 12:50:00
000736 2023-01-01 12:12:22
000736 2023-01-01 11:29:04
000736 2023-01-01 10:41:02
000736 2023-01-01 09:32:00
000736 2023-01-01 09:09:

000736 2022-12-29 15:23:57
000736 2022-12-29 15:15:27
000736 2022-12-29 15:07:41
000736 2022-12-29 15:04:51
000736 2022-12-29 15:01:04
000736 2022-12-29 14:57:43
000736 2022-12-29 14:55:30
000736 2022-12-29 14:55:04
000736 2022-12-29 14:53:49
000736 2022-12-29 14:53:02
000736 2022-12-29 14:51:49
000736 2022-12-29 14:49:41
000736 2022-12-29 14:48:26
000736 2022-12-29 14:43:54
000736 2022-12-29 14:40:47
000736 2022-12-29 14:40:30
000736 2022-12-29 14:38:05
000736 2022-12-29 14:37:51
000736 2022-12-29 14:32:28
000736 2022-12-29 14:28:57
222
000736 2022-12-29 14:27:54
000736 2022-12-29 14:23:04
000736 2022-12-29 14:17:45
000736 2022-12-29 14:12:12
000736 2022-12-29 14:08:43
000736 2022-12-29 14:07:45
000736 2022-12-29 14:02:23
000736 2022-12-29 14:01:11
000736 2022-12-29 13:59:28
000736 2022-12-29 13:59:22
000736 2022-12-29 13:58:54
000736 2022-12-29 13:58:32
000736 2022-12-29 13:53:28
000736 2022-12-29 13:52:53
000736 2022-12-29 13:52:42
000736 2022-12-29 13:49:44
000736 2022-12-29 13:48:

000736 2022-12-28 14:57:25
000736 2022-12-28 14:56:27
000736 2022-12-28 14:55:41
000736 2022-12-28 14:55:29
000736 2022-12-28 14:55:29
000736 2022-12-28 14:55:22
000736 2022-12-28 14:53:26
000736 2022-12-28 14:53:26
000736 2022-12-28 14:52:39
000736 2022-12-28 14:50:53
000736 2022-12-28 14:50:01
000736 2022-12-28 14:48:58
000736 2022-12-28 14:47:35
000736 2022-12-28 14:46:27
000736 2022-12-28 14:45:54
000736 2022-12-28 14:41:49
000736 2022-12-28 14:41:36
000736 2022-12-28 14:41:26
000736 2022-12-28 14:41:26
000736 2022-12-28 14:41:25
238


In [6]:
import pandas as pd
ls = []
lst = pd.read_excel('股票代码和名字.xlsx',dtype='str')['code']

for i in lst:
    sj = pd.read_csv(i+'.csv')
    sj['年月'] = sj['帖子时间'].str[0:7]
    cond = sj['年月'] == '2022-12'
    sj2 = pd.DataFrame(sj[cond]['帖子标题'])
    sj2 = sj2.reset_index()
    del sj2['index']
    sj2.rename(columns={"帖子标题": i.split('.')[0]},inplace=True)
    ls.append(sj2)

comments = pd.concat(ls,axis = 1)

In [7]:
comments

,000001,000002,000004,000005,000006,000011,000012,000017,000020,000021,...,688777,688779,688786,688787,688788,688789,688798,688800,688819,688981
0,$平安银行(SZ000001)$,万科2022年销售4202亿，与保利的差距达到371亿了，保利实现了弯道超车，超过碧桂园也就...,什么时候年报预告？别又来一遍预报高增长，年报后就活埋了[捂脸],公司压根就没打算摘帽，继续耗着吧！等实控人股权拍卖再说。,这只股会破八！一定会,$深物业A(SZ000011)$特发信息昨晚也出公告了！深国资对深投控又一大动作！深赛格、英...,$南玻Ａ(SZ000012)$我司即将参与南玻集团的两个项目招投标预祝顺利中标！,你好，请问贵公司的锂电池产品是否对外出售？都有哪些产品，是否可以应用到新能源汽车,深圳本地的资产重组标的,开门红,...,中控技术12月30日被沪股通减持2.16万股,$长远锂科(SH688779)$这家公司值不值得你买呢？不多说很多上市公司你可能没怎么了解你...,悦安新材：融资净偿还8.43万元，融资余额1.11亿元（12-30）,海天瑞声：融资净买入66.86万元，融资余额3604.98万元（12-30）,$科思科技(SH688788)$欺骗深交所！中天国富证券保荐科隆新能失败，章敬富、钟亚桢收监管函,宏华数科：融资净偿还32.41万元，融资余额6359.93万元（12-30）,艾为电子：融资净买入21.54万元，融资余额1.56亿元（12-30）,瑞可达12月30日获沪股通增持4.28万股,天能股份：融资余额2.33亿元，创历史新低（12-30）,中芯国际总结2022，展望2023：
1,$日丰股份(SZ002953)$继续吧$平安银行(SZ000001)$,"万科“裁员”传闻背后:南方区被上海区业绩赶超,郁亮“唱多”下业绩压力仍在",骂我的难道你们都是国华的忠诚散户亏损都在百分之五十以上吗哈哈。一说挂星退市跟仇人一样。谁都想...,几个小律师忙坏了,$深振业Ａ(SZ000006)$出货了，都小心点,$深物业A(SZ000011)$不减持是好还是不好,"$南玻Ａ(SZ000012)$000012,南玻A,凡是资金连断了欠债的走势都不行看下复星糸...",个人预感深中华A是2023年度深市最大的黑马；,总共才几个钱 不至于操碎了心呀,真正的垃圾是看不见的垃圾，到公司看看就知道什么叫垃圾了,...,中控技术：连续6日融资净偿还累计2934.85万元（12-30）,$长远锂科(SH688779)$$欣旺达(SZ300207)$这两个小垃圾选择老技术路线等着...,大宗交易：悦安新材成交1459.4万元，折价20.02%（12-30）,海天瑞声：融资净偿还86.35万元，融资余额3538.12万元（12-29）,$科思科技(SH688788)$牛掰了，上跌幅榜了，恭喜恭喜！,宏华数科：融资净买入5.44万元，融资余额6392.34万元（12-29）,虚拟的题材，巨客的科研投入一年能跌去六成剩4成股价的股票，不是一般的差，而是超级的差，不彻底...,瑞可达：连续3日融资净买入累计522.05万元（12-30）,$天能股份(SH688819)$董秘，我于22年七月下旬以旧换新，一组贵公司电池72v32a...,$中芯国际(SH688981)$维持买入，继续增持，2023加油干，走翻番行情……
2,什么时侯上16,就不退,恭喜st国华的散户今年亏损50.15%明年再接再厉争取退市。,谁知道年报预约时间啊,我已经满仓300万股深振业，成本5元左右,$深物业A(SZ000011)$央行货币政策委员会召开2022年第四季度例会：-要坚持稳字当...,南玻A12月30日获深股通增持78.86万股,$深中华A(SZ000017)$祝各位000017股友新年身体健康，股票大赚！,贵公的董秘还不如的学生,这只股跌的会超越所有人想象，不信走着看,...,$中控技术(SH688777)$垃圾股，扶不起的阿斗，真差劲的股票,$长远锂科(SH688779)$据报道，2022年股民人均亏6万多。只有期待2023再出发。,这要减持到明年3月份,转发,科思科技：连续4日融资净偿还累计224.96万元（12-30）,大宗交易：宏华数科成交1188.44万元，折价9.14%（12-29）,艾为电子：融资净买入128.26万元，融资余额1.56亿元（12-29）,$瑞可达(SH688800)$下周发利好[呲牙],被迫做五年股东,$中芯国际(SH688981)$明年与h股价格看齐
3,说点吓人的言论吧，都说明年股市会涨10%～20%，我的观点正好相反，明年会跌10%～20%，...,600122就是特例,你好，董秘。请问1.公司近期是否申请并且获批“一种大功率快充液冷电桩电缆”实用新,$ST星源(SZ000005)$兄弟们明年见。明年破历史新高,明天放假,$深物业A(SZ000011)$昨天11.51买入，深物业A期待来年我们春暖花开！,转眼一年就要过去了，明年南玻能涨起来吗,[赞][赞],《8.16见》,深科技12月30日被深股通减持6.36万股,...,垃圾。。。,大垃圾，大垃圾，垃圾垃圾垃圾,悦安新材：融资净偿还38.25万元，融资余额1.11亿元（12-29）,海天瑞声：融资净买入60.21万元，融资余额3624.47万元（12-28）,$科思科技(SH688788)$希望公司越来越好，我还想在这里养老[笑哭],专注数码喷印三十年 宏华数科市占率遥遥领先,艾为电子董事程剑涛个人名下持股减少32万股 涉及金额2813.44万元,疯了吗？,$天能股份(SH688819)$看看去年就知道了马上解禁中信证券又要套现了,有一家公司，它的企业口号就是：不怕你抄底，就怕你不玩！它就是大名鼎鼎的国之重器---中芯国际！
4,平安银行的两个区间统计,这个板块是这样的,$ST国华(SZ000004)$一个点就提有意思吗？祝你们卖飞。看看容积二六三。不买不卖。,$ST星源(SZ000005)$年底立案调查的嘞太多了。科林全是申请摘帽被打脸造假最无耻的一个股票,还有最后一张免税牌照，又会花落谁家？深圳横琴,深物业A：减持计划实施时间已经届满 深投控在减持计划期间未通过集中竞价交易或大宗交易方式减持...,玩长线短线现在都不是卖的时候，静候市场企稳,$深中华A(SZ000017)$今天拉到4个点的时候还以为自己套飞了，看样子没有走稳之前还是...,明年见了,$深科技(SZ000021)$昕天上午涨跌收阳，我已凌晨4点就发文……这时就不是“关注了”…...,...,一买一卖盈亏只在一念之间,长远锂科12月30日获沪股通增持11.02万股,大宗交易：悦安新材成交461.44万元，折价20.48%（12-29）,海天瑞声：融资净买入23.04万元，融资余额3564.25万元（12-27）,科思科技：连续3日融资净偿还累计200.22万元（12-29）,宏华数科：融资净买入107.12万元，融资余额6386.9万元（12-28）,艾为电子核心技术人员张忠个人名下持股减少21万股 涉及金额1846.32万元,擦。买早了,2022最后一天，天能雄起！2023年天能股份还是回到发行价之上吧！说太多不现实的，不如实际的。,$中芯国际(SH688981)$2022年已收官，还是要往前看，好的事就是探到了谷底，这一年...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
comments.to_csv('comments.csv',index=False)

In [6]:
comments.describe()

,000001,000002,000004,000005,000006,000011,000012,000017,000020,000021,...,688777,688779,688786,688787,688788,688789,688798,688800,688819,688981
count,1793,3098,416,535,10407,1824,1266,835,118,1298,...,136,553,86,95,110,49,134,107,419,2146
unique,1751,2987,412,526,10155,1779,1232,825,117,1250,...,136,544,85,94,108,47,132,105,412,2078
top,$平安银行(SZ000001)$,转发,$ST国华(SZ000004)$,$ST星源(SZ000005)$,$深振业Ａ(SZ000006)$,$深物业A(SZ000011)$,$南玻Ａ(SZ000012)$,$深中华A(SZ000017)$,今天可能要涨停,5.5,...,从图形上看，这分明就是不想跌,垃圾,悦安新材:2022年11月投资者关系活动记录表,$海天瑞声(SH688787)$氢能源概念股002438江苏神通大爆发,吐血了,宏华数科:杭州宏华数码科技股份有限公司投资者关系活动记录表(2022年11月),艾为电子：公司产品已逐步从消费电子领域转向AIOT、工业、汽车等领域,$瑞可达(SH688800)$12386*2投诉券商去，不需要他们做市，损害股民利益，建议撤...,不跑等死,垃圾企业千千万，中芯国际数第一。
freq,11,25,4,5,79,24,16,4,2,6,...,1,4,2,2,2,2,2,2,3,11


In [8]:
from joblib import dump, load

In [9]:
sgd_pipe= load('sgd_model.joblib')

In [10]:
MultinomialNB_model = load( 'MultinomialNB_pipe.joblib')

In [11]:
#定义分词函数
def cut_comment(comment_text):
    r = jieba.lcut(comment_text)
    joined_text = " ".join(r)
    return joined_text

In [12]:
import jieba
jieba.load_userdict("userdict.txt")

Building prefix dict from the default dictionary ...


2023-02-19 17:22:37,701 - jieba - DEBUG - Building prefix dict from the default dictionary ...


Loading model from cache C:\Users\86185\AppData\Local\Temp\jieba.cache


2023-02-19 17:22:37,708 - jieba - DEBUG - Loading model from cache C:\Users\86185\AppData\Local\Temp\jieba.cache


Loading model cost 1.095 seconds.


2023-02-19 17:22:38,803 - jieba - DEBUG - Loading model cost 1.095 seconds.


Prefix dict has been built successfully.


2023-02-19 17:22:38,805 - jieba - DEBUG - Prefix dict has been built successfully.


In [13]:
#定义函数统计情感词频
def count_sentiment_words(comment_text,sentiment):
    """
    基于情感词典，获取评论中对应的正面情感词。
    参数：
        comment_text：切词后的评论文本
        sentiment：是获取正面还是负面情感，1表示提取正面情感个数，0表示提取负面情感个数
    返回：
        评论中对应的情感词个数
    """
    comment_words_list = comment_text.split()
    
    if sentiment:
        return len([w for w in comment_words_list if w in positive_finance])
    else:
        return len([w for w in comment_words_list if w in negative_finance])

In [14]:
# 读取正面情感词汇
with open ("positive_finance.txt","r") as f:
    positive_finance = f.readlines()
    positive_finance = [w.strip() for w in positive_finance]

In [15]:
# 读取负面情感词汇
with open ("negative_finance.txt","r") as f:
    negative_finance = f.readlines()
    negative_finance = [w.strip() for w in negative_finance]

In [16]:
#批量计算市场情绪因子
indexlist = []
sumlist = []
meanlist = []
stdlist = []
comments_list = []
for i in comments.columns:
    tmp = pd.DataFrame()
    tmp["comment"] = [str(k) for k in comments[i].tolist() if type(k)==str]
    tmp["cutted_comment"] = tmp.comment.apply(cut_comment)
    tmp["positive_words_count"] = tmp.cutted_comment.apply(count_sentiment_words,args = (1,))
    tmp["negative_words_count"] = tmp.cutted_comment.apply(count_sentiment_words,args = (0,))
    sentiment_total_score = tmp.positive_words_count - tmp.negative_words_count
    tmp["sentiment_label"] = sentiment_total_score.apply(lambda x: 1 if x>=0 else -1)
    tmp["sentiment_sgd_clf"] = sgd_pipe.predict(tmp.cutted_comment)
    tmp["sentiment_MultinomialNB_clf"] = MultinomialNB_model.predict(tmp.cutted_comment)
    vote = tmp.sentiment_label + tmp.sentiment_sgd_clf + tmp.sentiment_MultinomialNB_clf 
    tmp["sentiment"] = vote.apply(lambda x: 1 if x>0 else -1)
    sen = list(tmp['sentiment'])
    print('第{}支股票{}的市场指数为{}，情绪总分为{}，均值为{}，标准差为{}'.format(list(comments.columns).index(i)+1, i, len(sen), sum(sen), np.mean(sen), np.std(sen)))
    indexlist.append(len(sen))
    sumlist.append(sum(sen))
    meanlist.append(np.mean(sen))
    stdlist.append(np.std(sen))
    words = []
    for w in tmp["cutted_comment"].tolist():
        words.extend(w.split())
    comments_list.extend(words)
    #print(len(comments_list))

第1支股票000001的市场指数为1793，情绪总分为-167，均值为-0.09313998884551032，标准差为0.9956530231349967
第2支股票000002的市场指数为3098，情绪总分为-942，均值为-0.3040671400903809，标准差为0.9526506045330874
第3支股票000004的市场指数为416，情绪总分为-208，均值为-0.5，标准差为0.8660254037844386
第4支股票000005的市场指数为535，情绪总分为-247，均值为-0.4616822429906542，标准差为0.8870453801847561
第5支股票000006的市场指数为10407，情绪总分为-6087，均值为-0.5848947823580283，标准差为0.811109174877436
第6支股票000011的市场指数为1824，情绪总分为-954，均值为-0.5230263157894737，标准差为0.8523165333323589
第7支股票000012的市场指数为1266，情绪总分为-672，均值为-0.5308056872037915，标准差为0.8474935530327713
第8支股票000017的市场指数为835，情绪总分为-441，均值为-0.5281437125748503，标准差为0.8491550028514547
第9支股票000020的市场指数为118，情绪总分为-56，均值为-0.4745762711864407，标准差为0.8802143845829683
第10支股票000021的市场指数为1298，情绪总分为-618，均值为-0.4761171032357473，标准差为0.8793818874677831
第11支股票000023的市场指数为77，情绪总分为-57，均值为-0.7402597402597403，标准差为0.6723209924958328
第12支股票000025的市场指数为1427，情绪总分为-715，均值为-0.5010511562718991，标准差为0.8654176672558708
第13支股票000026的市场指数为617，情绪总分为-313，均值为-0.5072933549432739，标准差为0.8617734342797982
第14支

第106支股票000610的市场指数为6820，情绪总分为-3864，均值为-0.5665689149560117，标准差为0.8240143594656391
第107支股票000612的市场指数为514，情绪总分为-250，均值为-0.48638132295719844，标准差为0.8737466501672012
第108支股票000616的市场指数为1798，情绪总分为-912，均值为-0.5072302558398221，标准差为0.861810575219792
第109支股票000617的市场指数为13124，情绪总分为-5382，均值为-0.4100883876866809，标准差为0.9120457851909293
第110支股票000620的市场指数为4317，情绪总分为-2263，均值为-0.5242066249710448，标准差为0.8515911074785049
第111支股票000625的市场指数为16215，情绪总分为-9223，均值为-0.5687943262411348，标准差为0.822479795761509
第112支股票000626的市场指数为591，情绪总分为-243，均值为-0.41116751269035534，标准差为0.9115598041313726
第113支股票000627的市场指数为529，情绪总分为-187，均值为-0.3534971644612476，标准差为0.9354355962426583
第114支股票000629的市场指数为4200，情绪总分为-2598，均值为-0.6185714285714285，标准差为0.7857285712987037
第115支股票000631的市场指数为574，情绪总分为-260，均值为-0.4529616724738676，标准差为0.8915299901123219
第116支股票000632的市场指数为2014，情绪总分为-928，均值为-0.46077457795431975，标准差为0.887517204515506
第117支股票000635的市场指数为460，情绪总分为-220，均值为-0.4782608695652174，标准差为0.8782178207271373
第118支股票000636的市场指数为1025，情绪总分为-525，均值为-

第210支股票000908的市场指数为1702，情绪总分为-848，均值为-0.4982373678025852，标准差为0.8670406710905503
第211支股票000909的市场指数为1125，情绪总分为-687，均值为-0.6106666666666667，标准差为0.7918877586010673
第212支股票000910的市场指数为241，情绪总分为-123，均值为-0.5103734439834025，标准差为0.8599528752649884
第213支股票000912的市场指数为525，情绪总分为-253，均值为-0.4819047619047619，标准差为0.8762236018582897
第214支股票000913的市场指数为199，情绪总分为-47，均值为-0.23618090452261306，标准差为0.9717091027354227
第215支股票000915的市场指数为175，情绪总分为-15，均值为-0.08571428571428572，标准差为0.9963197585235823
第216支股票000919的市场指数为2696，情绪总分为-1554，均值为-0.5764094955489614，标准差为0.8171609960350481
第217支股票000920的市场指数为996，情绪总分为-328，均值为-0.3293172690763052，标准差为0.9442193263686804
第218支股票000921的市场指数为329，情绪总分为-21，均值为-0.06382978723404255，标准差为0.9979607999624317
第219支股票000922的市场指数为366，情绪总分为-6，均值为-0.01639344262295082，标准差为0.9998656184902889
第220支股票000923的市场指数为519，情绪总分为-161，均值为-0.31021194605009633，标准差为0.9506674226709424
第221支股票000925的市场指数为459，情绪总分为-139，均值为-0.3028322440087146，标准差为0.9530438772630808
第222支股票000926的市场指数为1529，情绪总分为-685，均值为-0.44800523

第314支股票002048的市场指数为280，情绪总分为-84，均值为-0.3，标准差为0.9539392014169457
第315支股票002049的市场指数为2130，情绪总分为-1222，均值为-0.5737089201877934，标准差为0.8190592621397771
第316支股票002050的市场指数为481，情绪总分为-181，均值为-0.3762993762993763，标准差为0.926498127033563
第317支股票002052的市场指数为275，情绪总分为-113，均值为-0.4109090909090909，标准差为0.9116763235975059
第318支股票002053的市场指数为863，情绪总分为-365，均值为-0.42294322132097334，标准差为0.9061561849585523
第319支股票002054的市场指数为605，情绪总分为-167，均值为-0.27603305785123966，标准差为0.9611481420537077
第320支股票002056的市场指数为1499，情绪总分为-725，均值为-0.4836557705136758，标准差为0.8752583022450129
第321支股票002057的市场指数为788，情绪总分为-354，均值为-0.44923857868020306，标准差为0.8934118308067065
第322支股票002061的市场指数为821，情绪总分为-439，均值为-0.5347137637028014，标准差为0.8450332484031529
第323支股票002062的市场指数为1353，情绪总分为-589，均值为-0.43532889874353287，标准差为0.9002714867853713
第324支股票002064的市场指数为393，情绪总分为-197，均值为-0.5012722646310432，标准差为0.8652896143556015
第325支股票002065的市场指数为1059，情绪总分为-469，均值为-0.4428706326723324，标准差为0.8965855244852038
第326支股票002067的市场指数为284，情绪总分为-56，均值为-0.19718309859154928，标准差为

第418支股票002222的市场指数为374，情绪总分为-100，均值为-0.26737967914438504，标准差为0.9635912552429301
第419支股票002225的市场指数为273，情绪总分为-153，均值为-0.5604395604395604，标准差为0.8281953266556823
第420支股票002230的市场指数为2379，情绪总分为-725，均值为-0.30474989491382937，标准差为0.9524324131139228
第421支股票002231的市场指数为1223，情绪总分为-551，均值为-0.45053147996729354，标准差为0.8927605421155665
第422支股票002232的市场指数为523，情绪总分为-305，均值为-0.5831739961759083，标准差为0.8123472719128328
第423支股票002233的市场指数为224，情绪总分为-12，均值为-0.05357142857142857，标准差为0.9985640200011295
第424支股票002234的市场指数为1080，情绪总分为-480，均值为-0.4444444444444444，标准差为0.8958064164776167
第425支股票002235的市场指数为5040，情绪总分为-2390，均值为-0.4742063492063492，标准差为0.8804137313629236
第426支股票002237的市场指数为469，情绪总分为-207，均值为-0.44136460554371004，标准差为0.8973278581283682
第427支股票002238的市场指数为550，情绪总分为-246，均值为-0.44727272727272727，标准差为0.8943976226701502
第428支股票002239的市场指数为1092，情绪总分为-654，均值为-0.5989010989010989，标准差为0.8008229977560937
第429支股票002240的市场指数为3117，情绪总分为-1643，均值为-0.5271094000641643，标准差为0.8497974349008102
第430支股票002241的市场指数为16030，情绪总分为-9884，均值为

...

第2293支股票603565的市场指数为203，情绪总分为-61，均值为-0.30049261083743845，标准差为0.9537841426822421
第2294支股票603566的市场指数为68，情绪总分为4，均值为0.058823529411764705，标准差为0.9982683969692436
第2295支股票603568的市场指数为541，情绪总分为-221，均值为-0.40850277264325324，标准差为0.9127570787141421
第2296支股票603577的市场指数为370，情绪总分为-228，均值为-0.6162162162162163，标准差为0.7875770278976967
第2297支股票603578的市场指数为51，情绪总分为-23，均值为-0.45098039215686275，标准差为0.8925338569992974
第2298支股票603579的市场指数为214，情绪总分为-56，均值为-0.2616822429906542，标准差为0.9651540828817854
第2299支股票603580的市场指数为167，情绪总分为-83，均值为-0.49700598802395207，标准差为0.8677471105502658
第2300支股票603583的市场指数为402，情绪总分为-140，均值为-0.3482587064676617，标准差为0.9373984602982187
第2301支股票603585的市场指数为36，情绪总分为-18，均值为-0.5，标准差为0.8660254037844386
第2302支股票603586的市场指数为64，情绪总分为-30，均值为-0.46875，标准差为0.8833308765689106
第2303支股票603589的市场指数为2546，情绪总分为142，均值为0.05577376276512176，标准差为0.9984434322419172
第2304支股票603595的市场指数为285，情绪总分为-165，均值为-0.5789473684210527，标准差为0.8153649149910351
第2305支股票603598的市场指数为301，情绪总分为-169，均值为-0.5614617940199336，标准差为0.82750266093

第2399支股票603893的市场指数为727，情绪总分为-417，均值为-0.5735900962861072，标准差为0.8191424793297527
第2400支股票603895的市场指数为175，情绪总分为-59，均值为-0.33714285714285713，标准差为0.9414535006454389
第2401支股票603896的市场指数为273，情绪总分为-49，均值为-0.1794871794871795，标准差为0.983760312474404
第2402支股票603898的市场指数为58，情绪总分为-16，均值为-0.27586206896551724，标准差为0.961197232052852
第2403支股票603899的市场指数为164，情绪总分为-6，均值为-0.036585365853658534，标准差为0.9993305314085794
第2404支股票603901的市场指数为113，情绪总分为-37，均值为-0.3274336283185841，标准差为0.9448741815956913
第2405支股票603906的市场指数为1339，情绪总分为-589，均值为-0.4398805078416729，标准差为0.8980563116091062
第2406支股票603908的市场指数为684，情绪总分为-300，均值为-0.43859649122807015，标准差为0.8986841034993472
第2407支股票603909的市场指数为91，情绪总分为-41，均值为-0.45054945054945056，标准差为0.8927514730369188
第2408支股票603915的市场指数为114，情绪总分为-4，均值为-0.03508771929824561，标准差为0.9993842363948152
第2409支股票603916的市场指数为191，情绪总分为-49，均值为-0.25654450261780104，标准差为0.9665324196200483
第2410支股票603917的市场指数为220，情绪总分为-96，均值为-0.43636363636363634，标准差为0.8997704023024453
第2411支股票603918的市场指数为726，情绪总分为-418，均值为-0.57575

第2503支股票605378的市场指数为332，情绪总分为-168，均值为-0.5060240963855421，标准差为0.8625193411612261
第2504支股票605388的市场指数为395，情绪总分为37，均值为0.09367088607594937，标准差为0.9956032166991762
第2505支股票605398的市场指数为244，情绪总分为-98，均值为-0.4016393442622951，标准差为0.9157979237476758
第2506支股票605399的市场指数为351，情绪总分为-101，均值为-0.28774928774928776，标准差为0.9577057728758753
第2507支股票605499的市场指数为222，情绪总分为-48，均值为-0.21621621621621623，标准差为0.9763455063885643
第2508支股票605507的市场指数为304，情绪总分为-150，均值为-0.4934210526315789，标准差为0.8697905867620922
第2509支股票605555的市场指数为220，情绪总分为-88，均值为-0.4，标准差为0.9165151389911679
第2510支股票605566的市场指数为74，情绪总分为-34，均值为-0.4594594594594595，标准差为0.8881987419002695
第2511支股票605567的市场指数为535，情绪总分为-245，均值为-0.45794392523364486，标准差为0.8889810804182515
第2512支股票605577的市场指数为447，情绪总分为-277，均值为-0.6196868008948546，标准差为0.7848492013098446
第2513支股票605589的市场指数为1221，情绪总分为-569，均值为-0.46601146601146604，标准差为0.8847786805443745
第2514支股票605599的市场指数为179，情绪总分为-101，均值为-0.5642458100558659，标准差为0.8256068470127895
第2515支股票688001的市场指数为62，情绪总分为6，均值为0.0967741935483871，标准差为0

第2610支股票688195的市场指数为54，情绪总分为28，均值为0.5185185185185185，标准差为0.8550663985640886
第2611支股票688196的市场指数为83，情绪总分为25，均值为0.30120481927710846，标准差为0.9535594668631027
第2612支股票688197的市场指数为92，情绪总分为16，均值为0.17391304347826086，标准差为0.9847610132961846
第2613支股票688198的市场指数为63，情绪总分为29，均值为0.4603174603174603，标准差为0.8877543780387025
第2614支股票688199的市场指数为144，情绪总分为26，均值为0.18055555555555555，标准差为0.9835647875752896
第2615支股票688200的市场指数为113，情绪总分为17，均值为0.1504424778761062，标准差为0.9886187641606329
第2616支股票688201的市场指数为56，情绪总分为34，均值为0.6071428571428571，标准差为0.794592695045964
第2617支股票688203的市场指数为49，情绪总分为9，均值为0.1836734693877551，标准差为0.9829873125544731
第2618支股票688206的市场指数为242，情绪总分为-84，均值为-0.34710743801652894，标准差为0.9378253709905706
第2619支股票688208的市场指数为247，情绪总分为-93，均值为-0.3765182186234818，标准差为0.9264092136008796
第2620支股票688209的市场指数为84，情绪总分为6，均值为0.07142857142857142，标准差为0.9974457174120671
第2621支股票688211的市场指数为122，情绪总分为-8，均值为-0.06557377049180328，标准差为0.9978477241661117
第2622支股票688213的市场指数为68，情绪总分为24，均值为0.35294117647058826，标准差为0.935645512975698

第2717支股票688488的市场指数为284，情绪总分为40，均值为0.14084507042253522，标准差为0.9900316490585901
第2718支股票688501的市场指数为49，情绪总分为23，均值为0.46938775510204084，标准差为0.8829921490932218
第2719支股票688508的市场指数为118，情绪总分为6，均值为0.05084745762711865，标准差为0.9987064313665245
第2720支股票688510的市场指数为80，情绪总分为44，均值为0.55，标准差为0.8351646544245033
第2721支股票688511的市场指数为24，情绪总分为16，均值为0.6666666666666666，标准差为0.7453559924999298
第2722支股票688513的市场指数为121，情绪总分为-17，均值为-0.14049586776859505，标准差为0.9900812649171529
第2723支股票688516的市场指数为242，情绪总分为52，均值为0.21487603305785125，标准差为0.976641331511892
第2724支股票688518的市场指数为121，情绪总分为-23，均值为-0.19008264462809918，标准差为0.9817680928871075
第2725支股票688521的市场指数为272，情绪总分为-110，均值为-0.40441176470588236，标准差为0.9145770194835828
第2726支股票688551的市场指数为50，情绪总分为14，均值为0.28，标准差为0.96
第2727支股票688553的市场指数为401，情绪总分为-159，均值为-0.39650872817955113，标准差为0.9180309518079632
第2728支股票688555的市场指数为963，情绪总分为-505，均值为-0.5244029075804777，标准差为0.8514702522819815
第2729支股票688556的市场指数为366，情绪总分为-30，均值为-0.08196721311475409，标准差为0.9966350264636501
第2730支股票688557的市场指数为69，

<font size="6">**数据整合**

In [17]:
basic_stock = pd.read_excel('基本面2022-3+股价数据.xlsx')
basic_stock['code'] = basic_stock.apply(lambda x:'0'*(6-len(str(x[0])))+str(x[0]),axis=1)

In [18]:
df = basic_stock.copy()
df['sen_index'] = indexlist
df['sen_score'] = sumlist
df['sen_mean'] = meanlist
df['sen_std'] = stdlist
df

,code,name,area,industry,market,list_date,eps,eps_yoy,bvps,roe,...,Volatility_15,Volatility_21,Volatility_30,Volatility_60,yield,label,sen_index,sen_score,sen_mean,sen_std
0,000001,平安银行,深圳,银行,主板,19910403,1.78,27.14,18.32,10.15,...,-0.038197,0.095625,0.488579,-0.174041,0.020360,1,1793,-167,-0.093140,0.995653
1,000002,万科A,深圳,全国地产,主板,19910129,1.47,2.35,NaN,7.12,...,0.410888,1.492037,1.289951,1.546284,-0.011432,0,3098,-942,-0.304067,0.952651
2,000004,ST国华,深圳,软件服务,主板,19910114,-0.38,-281.88,NaN,-6.17,...,0.586980,-0.475211,0.376238,0.779977,0.010571,1,416,-208,-0.500000,0.866025
3,000005,ST星源,深圳,环境保护,主板,19901210,0.01,-94.82,NaN,0.73,...,-0.803684,-0.234872,-0.404787,-0.545122,0.005435,1,535,-247,-0.461682,0.887045
4,000006,深振业A,深圳,区域地产,主板,19920427,0.15,-64.21,NaN,2.61,...,-0.518529,-0.496677,-0.280033,-0.623720,0.014901,1,10407,-6087,-0.584895,0.811109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2794,688789,宏华数科,浙江,专用机械,科创板,20210708,2.45,-7.89,NaN,11.86,...,0.316911,0.199722,-0.147357,1.353955,-0.022152,0,49,31,0.632653,0.774435
2795,688798,艾为电子,上海,半导体,科创板,20210816,0.33,-78.29,NaN,1.45,...,0.520592,0.800187,-0.276064,1.049499,0.009927,1,134,2,0.014925,0.999889
2796,688800,瑞可达,江苏,元器件,科创板,20210722,1.82,121.95,NaN,18.24,...,0.853940,0.064290,0.946856,0.515999,0.041214,1,107,59,0.551402,0.834240
2797,688819,天能股份,浙江,电气设备,科创板,20210118,1.47,32.43,NaN,11.14,...,-0.106737,0.730579,-0.059349,0.860177,0.009397,1,419,-149,-0.355609,0.934635


In [19]:
df.to_excel('数据汇总.xlsx',index=False)

In [20]:
df.shape

(2799, 90)

In [21]:
df.info() #查看数据基本信息

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2799 entries, 0 to 2798
Data columns (total 90 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   code                   2799 non-null   object 
 1   name                   2799 non-null   object 
 2   area                   2799 non-null   object 
 3   industry               2799 non-null   object 
 4   market                 2799 non-null   object 
 5   list_date              2799 non-null   int64  
 6   eps                    2799 non-null   float64
 7   eps_yoy                2699 non-null   float64
 8   bvps                   22 non-null     float64
 9   roe                    2790 non-null   float64
 10  epcf                   14 non-null     float64
 11  net_profits            2799 non-null   float64
 12  profits_yoy            2799 non-null   float64
 13  distrib                13 non-null     object 
 14  report_date            2799 non-null   object 
 15  net_

In [22]:
df.describe() #描述性统计

,list_date,eps,eps_yoy,bvps,roe,epcf,net_profits,profits_yoy,net_profit_ratio,gross_profit_rate,...,Volatility_15,Volatility_21,Volatility_30,Volatility_60,yield,label,sen_index,sen_score,sen_mean,sen_std
count,2.799000e+03,2799.000000,2699.000000,22.000000,2790.000000,14.000000,2.799000e+03,2799.00000,2691.000000,2691.000000,...,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000
mean,2.011610e+07,0.572315,-25.898951,10.390909,5.947441,-0.027143,8.752336e+04,-8.67343,-4.980145,29.791550,...,0.152400,0.145596,0.174039,0.784931,-0.008850,0.287603,840.040372,-357.486602,-0.331793,0.908596
std,8.987134e+04,1.578429,960.620823,6.583100,12.459442,5.024325,6.994897e+05,1170.71422,586.048347,19.994751,...,1.000441,0.792784,0.814360,1.149763,0.038575,0.452726,1784.677055,917.527411,0.244040,0.069532
min,1.990121e+07,-3.870000,-32433.330000,3.740000,-128.880000,-15.870000,-1.167312e+06,-38054.33000,-29597.210000,-190.945800,...,-1.000000,-1.000000,-1.000000,-1.000000,-0.149551,0.000000,10.000000,-27757.000000,-0.900000,0.416598
25%,2.005053e+07,0.070000,-51.050000,5.562500,1.990000,0.122500,3.025685e+03,-46.16000,2.480000,16.267800,...,-0.304504,-0.304808,-0.319351,0.020359,-0.028929,0.000000,186.000000,-360.500000,-0.495382,0.865942
50%,2.015013e+07,0.290000,-11.110000,8.355000,5.640000,0.690000,1.069340e+04,-2.88000,7.710000,25.914400,...,-0.017541,-0.019309,-0.009652,0.537396,-0.014025,0.000000,394.000000,-141.000000,-0.387097,0.918153
75%,2.020021e+07,0.660000,22.220000,12.632500,9.827500,1.832500,3.500796e+04,30.82000,15.250000,39.704350,...,0.409118,0.397823,0.422294,1.228936,0.004275,1.000000,859.500000,-48.000000,-0.237286,0.963927
max,2.022103e+07,35.340000,21852.170000,31.000000,106.670000,5.750000,2.658220e+07,28652.39000,909.540000,100.000000,...,36.972897,22.491265,9.411899,17.096272,0.331561,1.000000,49637.000000,1242.000000,0.909091,1.000000
